# Compose TensorFlow Decision Forests and Neural Networks

This notebook shows you how to compose multiple TensorFlow [Decision Forests](https://www.tensorflow.org/decision_forests) and Neural Networks together using a common preprocessing layer and the [Keras functional API](https://keras.io/guides/functional_api/).

In [ ]:
from graphviz import Source

Source("""
digraph G {
  raw_data [label="Input features"];
  preprocess_data [label="Learnable NN pre-processing", shape=rect];

  raw_data -> preprocess_data

  subgraph cluster_0 {
    color=grey;
    a1[label="NN layer", shape=rect];
    b1[label="NN layer", shape=rect];
    a1 -> b1;
    label = "Model #1";
  }

   subgraph cluster_1 {
    color=grey;
    a2[label="NN layer", shape=rect];
    b2[label="NN layer", shape=rect];
    a2 -> b2;
    label = "Model #2";
  }

  subgraph cluster_2 {
    color=grey;
    a3[label="GradientBoost", shape=rect];
    label = "Model #3";
  }

  subgraph cluster_3 {
    color=grey;
    a4[label="Random Forest", shape=rect];
    label = "Model #4";
  }
  

  preprocess_data -> a1;
  preprocess_data -> a2;
  preprocess_data -> a3;
  preprocess_data -> a4;

  b1  -> aggr;
  b2  -> aggr;
  a3 -> aggr;
  a4 -> aggr;
  
  aggr [label="Aggregation (mean)", shape=rect]
  aggr -> predictions
}
""")

In [ ]:
!pip install tensorflow_decision_forests -U --quiet

In [ ]:
import os, glob, math, cv2, gc, logging
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import tensorflow_decision_forests as tfdf
import warnings

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.utils import plot_model

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

warnings.filterwarnings("ignore")
tf.autograph.set_verbosity(0)
logging.getLogger("tensorflow").setLevel(logging.ERROR)
print(tf.__version__)

# Load and Preprocess Data

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-jan-2022/train.csv').set_index("row_id")
test = pd.read_csv('../input/tabular-playground-series-jan-2022/test.csv').set_index("row_id")

sample_submission = pd.read_csv("../input/tabular-playground-series-jan-2022/sample_submission.csv")

In [ ]:
train['date'] = pd.to_datetime(train['date'])
train['year'] = train['date'].dt.year
train['month'] = train['date'].dt.month
train['day'] = train['date'].dt.day
train['dayofweek'] = train['date'].dt.dayofweek

test['date'] = pd.to_datetime(test['date'])
test['year'] = test['date'].dt.year
test['month'] = test['date'].dt.month
test['day'] = test['date'].dt.day
test['dayofweek'] = test['date'].dt.dayofweek

In [ ]:
train["month"] = train["month"].astype('category')
train["dayofweek"] = train["dayofweek"].astype('category')
train["day"] = train["day"].astype('category')

test["month"] = test["month"].astype('category')
test["dayofweek"] = test["dayofweek"].astype('category')
test["day"] = test["day"].astype('category')

# Some plots to get insight from data

In [ ]:
fig, ax = plt.subplots(1, 4, figsize=(25, 8))
sns.boxplot(data=train, x='country', y='num_sold', ax=ax[0])
sns.boxplot(data=train, x='store', y='num_sold', ax=ax[1])
sns.boxplot(data=train, x='product', y='num_sold', ax=ax[2])
sns.boxplot(data=train, x='year', y='num_sold', ax=ax[3])
plt.show()

In [ ]:
fig, ax = plt.subplots(2, 3, figsize=(25, 12))
sns.violinplot(data=train, x='country', y='num_sold', ax=ax[0][0])
sns.violinplot(data=train, x='store', y='num_sold', ax=ax[0][1])
sns.violinplot(data=train, x='product', y='num_sold', ax=ax[0][2])
sns.violinplot(data=train, x='year', y='num_sold', ax=ax[1][0])
sns.violinplot(data=train, x='month', y='num_sold', ax=ax[1][1])
sns.violinplot(data=train, x='dayofweek', y='num_sold', ax=ax[1][2])
plt.show()

# Add More Features

 - Thanks to [@carlmcbrideellis](https://www.kaggle.com/carlmcbrideellis) for the [GDP dataset](https://www.kaggle.com/carlmcbrideellis/gdp-20152019-finland-norway-and-sweden).
 - Thanks to [@vpallares](https://www.kaggle.com/vpallares) for the [Holidays dataset](https://www.kaggle.com/vpallares/public-and-unofficial-holidays-nor-fin-swe-201519).

## GDP

In [ ]:
GDP_data = pd.read_csv("../input/gdp-20152019-finland-norway-and-sweden/GDP_data_2015_to_2019_Finland_Norway_Sweden.csv").set_index("year")
GDP_data.columns = ['Finland', 'Norway', 'Sweden']
GDP_dictionary = GDP_data.unstack().to_dict()
train["GDP"] = train.set_index(['country','year']).index.map(GDP_dictionary.get)
test["GDP"] = test.set_index(['country','year']).index.map(GDP_dictionary.get)

sc = MinMaxScaler()
train["GDP"] = sc.fit_transform(train["GDP"].values.reshape(-1,1))
test["GDP"] = sc.transform(test["GDP"].values.reshape(-1,1))

## Holidays

In [ ]:
holidays_data = pd.read_csv("../input/public-and-unofficial-holidays-nor-fin-swe-201519/holidays.csv")
train["is_holiday"] = train.date.isin(holidays_data["date"]).astype(np.uint8)
test["is_holiday"] = test.date.isin(holidays_data["date"]).astype(np.uint8)

## Remove Redundant Columns

In [ ]:
train_date = train["date"]
test_date = test["date"]
train.drop(["date", "year"], inplace=True, axis=1)
test.drop(["date", "year"], inplace=True, axis=1)

## One-Hoe Encode Categorical Columns

In [ ]:
ohe_cols = ["country","store","product", "month", "dayofweek", "day"]

train = pd.get_dummies(train, columns=["month"], prefix="o_month")
test = pd.get_dummies(test, columns=["month"], prefix="o_month")

train = pd.get_dummies(train, columns=["day"], prefix="o_day")
test = pd.get_dummies(test, columns=["day"], prefix="o_day")

train = pd.get_dummies(train, columns=["dayofweek"], prefix="o_week")
test = pd.get_dummies(test, columns=["dayofweek"], prefix="o_week")

train = pd.get_dummies(train, columns=["country","store","product"], prefix="ohe")
test = pd.get_dummies(test, columns=["country","store","product"], prefix="ohe")

train.head()

# Split Data and Make Dataset

In [ ]:
x_train, x_valid, y_train, y_valid = train_test_split(train[train.columns.tolist()[1:]], train.num_sold, test_size=0.2, random_state=42)
print(f"x_train  shape: {x_train.shape}")
print(f"y_train  shape: {y_train.shape}\n")
print(f"x_valid  shape: {x_valid.shape}")
print(f"y_valid  shape: {y_valid.shape}")

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train.values.astype("float32"), y_train.values.astype("float32"))).batch(64)
valid_dataset = tf.data.Dataset.from_tensor_slices((x_valid.values.astype("float32"), y_valid.values.astype("float32"))).batch(64)

# Create Model

In [ ]:
def seed_advanced_argument(seed):
    return tfdf.keras.AdvancedArguments(yggdrasil_training_config=tfdf.keras.core.YggdrasilTrainingConfig(random_seed=seed))

inputs = layers.Input(shape=(x_train.shape[-1],))

# learnable pre-processing
x = tf.keras.layers.Dense(1024, activation=tf.nn.relu6)(inputs)
preprocess_features = tf.keras.layers.Dense(200, activation=tf.nn.relu6)(x)

preprocessor = tf.keras.Model(inputs, preprocess_features)

# Model #1: Neural Network 1
m1_z1 = tf.keras.layers.Dense(256, activation=tf.nn.relu6)(preprocess_features)
m1_z1 = tf.keras.layers.Dropout(0.2)(m1_z1)
m1_z1 = tf.keras.layers.Dense(128, activation=tf.nn.relu6)(m1_z1)
m1_pred = tf.keras.layers.Dense(1)(m1_z1)

# Model #2: Neural Network 2
m2_z1 = tf.keras.layers.Dense(128, activation=tf.nn.swish)(preprocess_features)
m2_z1 = tf.keras.layers.Dropout(0.2)(m2_z1)
m2_z1 = tf.keras.layers.Dense(64, activation=tf.nn.swish)(m2_z1)
m2_pred = tf.keras.layers.Dense(1)(m2_z1)

# Model #3: DF Gradient Boost Trees
model_3 = tfdf.keras.GradientBoostedTreesModel(num_trees=10000, 
                                               advanced_arguments=seed_advanced_argument(1234), 
                                               task=tfdf.keras.Task.REGRESSION)
m3_pred = model_3(preprocess_features)

# Model #4: DF Random Forest
model_4 = tfdf.keras.RandomForestModel(num_trees=2000,
                                               advanced_arguments=seed_advanced_argument(2022), 
                                               task=tfdf.keras.Task.REGRESSION)
m4_pred = model_4(preprocess_features)

mean_nn_only = tf.reduce_mean(tf.stack([m1_pred, m2_pred], axis=0), axis=0)
mean_nn_and_df = tf.reduce_mean(tf.stack([m1_pred, m2_pred, m3_pred, m4_pred], axis=0), axis=0)


ensemble_nn_only = tf.keras.models.Model(inputs, mean_nn_only)
ensemble_nn_and_df = tf.keras.models.Model(inputs, mean_nn_and_df)

In [ ]:
plot_model(ensemble_nn_and_df, to_file="model.png", show_shapes=True)

# Train Neural Network Models 🧠

In [ ]:
def smape(y_true, y_pred):
    denominator = (y_true + tf.abs(y_pred)) / 200.0
    diff = tf.abs(y_true - y_pred) / denominator
    diff = tf.where(denominator==0, 0.0, diff)
    return tf.reduce_mean(diff)

cb_es = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=6, mode="min", restore_best_weights=True, verbose=1)
cb_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=2, mode="min", min_lr=1e-6, verbose=1)

ensemble_nn_only.compile(optimizer=tf.keras.optimizers.Adam(0.001), loss=smape, metrics=[smape])

history = ensemble_nn_only.fit(train_dataset, 
                               epochs=100,
                               validation_data=valid_dataset,
                               callbacks=[cb_es, cb_lr],
                               verbose=2)

# Train TF Decision Forest Models (TFDF) 🎄 🌴 🌳

TFDF does not support (yet) GPUs and TPUs. Therefore, the training process may be slow. You can get much higher accuracy by amending the decision trees arguments (for example `num_trees` or increasing pre-processed features).

In [ ]:
%%time
train_dataset_with_preprocessing = train_dataset.map(lambda x,y: (preprocessor(x), y)).unbatch().batch(1024)
valid_dataset_with_preprocessing = valid_dataset.map(lambda x,y: (preprocessor(x), y)).unbatch().batch(1024)

model_3.compile([smape])
model_4.compile([smape])

print("\n-------------#3 Training TFDF Gradinet Boost Trees -------------\n")
model_3.fit(train_dataset_with_preprocessing)
evaluation_df3_only = model_3.evaluate(valid_dataset_with_preprocessing, return_dict=True)

print("\n-------------#4 Training TFDF Random Forest -------------\n")
model_4.fit(train_dataset_with_preprocessing)
evaluation_df4_only = model_4.evaluate(valid_dataset_with_preprocessing, return_dict=True)

# Evaluate Models

In [ ]:
print("Model Neural Network Only:")
evaluation_nn_only = ensemble_nn_only.evaluate(valid_dataset)
print("\nModel TFDF Gradient Boost Trees Only:")
evaluation_df3_only = model_3.evaluate(valid_dataset_with_preprocessing, return_dict=True)
print("\nModel TFDF Random Forest Only:")
evaluation_df4_only = model_4.evaluate(valid_dataset_with_preprocessing, return_dict=True)

ensemble_nn_and_df.compile(loss=smape, metrics=[smape])
print("\nEnsembeled Neural Networks and TFDF Models:")
evaluation_nn_and_df = ensemble_nn_and_df.evaluate(valid_dataset, return_dict=True)

In [ ]:
sample_train_data = train.loc[(train["ohe_Finland"]==1) & (train["ohe_KaggleMart"]==1) & (train["ohe_Kaggle Hat"]==1)]
p = ensemble_nn_and_df.predict(sample_train_data[sample_train_data.columns.tolist()[1:]]).astype('int32')

In [ ]:
plt.figure(figsize=(25,8))
sns.lineplot(y=sample_train_data["num_sold"].values, x=train_date.unique())
sns.lineplot(y=p.ravel(), x=train_date.unique())
plt.legend(["Actual", "Preds"])
plt.title("Num Sold for Hat at the Mart in Finland (Prediction VS Actual Selling)")
plt.show()

# Create Submission

In [ ]:
preds = ensemble_nn_and_df.predict(test).astype('int32')
sample_submission["num_sold"] = preds.ravel()
sample_submission.to_csv("submission.csv", index=False)
sample_submission

# Plot Test Predictions

In [ ]:
p_1 = ensemble_nn_and_df.predict(test.loc[(test["ohe_Finland"]==1) & (test["ohe_KaggleMart"]==1) & (test["ohe_Kaggle Hat"]==1)]).astype('int32')
p_2 = ensemble_nn_and_df.predict(test.loc[(test["ohe_Finland"]==1) & (test["ohe_KaggleMart"]==1) & (test["ohe_Kaggle Mug"]==1)]).astype('int32')
p_3 = ensemble_nn_and_df.predict(test.loc[(test["ohe_Finland"]==1) & (test["ohe_KaggleMart"]==1) & (test["ohe_Kaggle Sticker"]==1)]).astype('int32')

plt.figure(figsize=(25,8))
sns.lineplot(y=p_1.ravel(), x=test_date.unique())
sns.lineplot(y=p_2.ravel(), x=test_date.unique())
sns.lineplot(y=p_3.ravel(), x=test_date.unique())
plt.title("Prediction for the Mart store in Finland")
plt.legend(["Hat", "Mug", "Sticker"])
plt.show()

Source: [Composing Decision Forest and Neural Network models](https://www.tensorflow.org/decision_forests/tutorials/model_composition_colab)